In [4]:
import os
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

#### Step 1: Find and download a dataset for supervised learning. 
Loading ecommerce data from kaggle (https://www.kaggle.com/prachi13/customer-analytics)


In [11]:
folder = "https://github.com/beyenidogan/AdvData/blob/main/Project1/"
file_name="ecommerce_shipping.csv"
#file_url=folder+file_name
file_url="https://github.com/beyenidogan/AdvData/blob/main/Project1/ecommerce_shipping.csv"

#def load_shipping_data(path=file_url): 
#    return pd.read_csv(path)

shipping = pd.read_csv(file_url,error_bad_lines=False,header=None)

shipping.head()

b'Skipping line 84: expected 1 fields, saw 2\nSkipping line 103: expected 1 fields, saw 253\nSkipping line 104: expected 1 fields, saw 2\nSkipping line 119: expected 1 fields, saw 3\nSkipping line 169: expected 1 fields, saw 3\nSkipping line 188: expected 1 fields, saw 6\nSkipping line 214: expected 1 fields, saw 4\nSkipping line 218: expected 1 fields, saw 2\nSkipping line 231: expected 1 fields, saw 2\nSkipping line 232: expected 1 fields, saw 2\nSkipping line 238: expected 1 fields, saw 2\nSkipping line 241: expected 1 fields, saw 2\nSkipping line 249: expected 1 fields, saw 4\nSkipping line 255: expected 1 fields, saw 2\nSkipping line 260: expected 1 fields, saw 2\nSkipping line 261: expected 1 fields, saw 2\nSkipping line 262: expected 1 fields, saw 2\nSkipping line 263: expected 1 fields, saw 2\nSkipping line 264: expected 1 fields, saw 2\nSkipping line 270: expected 1 fields, saw 2\nSkipping line 271: expected 1 fields, saw 2\nSkipping line 272: expected 1 fields, saw 2\nSkippin

,0
0,<!DOCTYPE html>
1,"<html lang=""en"" data-color-mode=""auto"" data-li..."
2,<head>
3,"<meta charset=""utf-8"">"
4,"<link rel=""dns-prefetch"" href=""https://githu..."


Step 2: Divide into a training set and a testing set. In a Jupyter notebook, use scikitlearn to divide you data into training and testing sets. Make sure that the testing and training sets are balanced in terms of target classes

Step 3: Explore your training set. In a Jupyter notebook, import your data into a Pandas data frame and use the following pandas functions to explore your data

DataFrame.info()
DataFrame.describe()

Step 4: Data cleaning. Address any missing values in your training set. Include the code in your Jupyter notebook and create a second, cleaned, version of your dataset. Then apply the same procedure to you test set (if you are putting in replacement values use IMPUTER in scikitlearn).

Recall from the Hands on Machine Learning book (p. 60) that some options are
“Get rid of the corresponding samples.”
“Get rid of the whole attribute (column).”
“Set the values to some value (zero, the mean, the median, etc.).”

Step 5: Visualize the data in your training set. At a minimum, use the following pandas functions to visualize the data in your Jupyter notebook.

DataFrame.hist
plotting.scatter_matrix()

Step 6: Apply transformations to your data. In your Jupetyr notebook apply, squaring, cubing, logarithmic, and exponentials transformations to two features in your dataset. Plot the histograms and scatter matrices of the resultant data.